In [1]:
import numpy as np
import pandas as pd 
from pathlib import Path
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

from engine_hms_model import CustomDataset, JobConfig, ModelConfig, CustomVITMAE
from engine_hms_trainer import load_kaggle_data, TARGETS, TARGETS_PRED, BRAIN_ACTIVITY, DEVICE

/home/shiyi/miniconda3/envs/kaggle/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/shiyi/miniconda3/envs/kaggle/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
train_easy, train_hard, all_specs, all_eegs = load_kaggle_data(JobConfig.PATHS, JobConfig.ENTROPY_SPLIT)

In [3]:
def reshape_mae_input(x): #<- (N, C, H, W)
    x = torch.stack(x, dim=0)
    concat_p1 = torch.cat(torch.chunk(x[:, :4, :, :], 4, dim=1), dim=2)
    concat_p2 = torch.cat(torch.chunk(x[:, 4:, :, :], 4, dim=1), dim=2)
    x_concat = torch.cat((concat_p1, concat_p2), dim=3)
   
    resized = F.interpolate(x_concat, size=(224, 224), mode='bilinear', align_corners=False)
    stacked = resized.repeat(1, 3, 1, 1)
    
    return stacked

In [21]:
# Instantiate your model

ModelConfig.MAE_HIDDEN_DROPOUT_PROB = 0.1
ModelConfig.MAE_ATTENTION_DROPOUT_PROB = 0.1

model = CustomVITMAE(ModelConfig)

train_dataset = CustomDataset(train_easy, TARGETS, ModelConfig, all_specs, all_eegs)

# Test the model with a single batch
X, y = train_dataset[0]
logits = model(X.unsqueeze(0))
print(logits.shape)  # Expected to be [batch_size, num_classes]

torch.Size([1, 6])
